In [70]:
# Importing for system files
import os
import sys
import csv
import pprint

In [71]:
# Loading environmental variables 
from dotenv import load_dotenv
load_dotenv()

True

In [72]:
# Import System Files
dir_path = os.path.dirname(os.path.realpath('__file__'))
sys.path.append(dir_path + '/lib/')  
from process_work import process_work

In [73]:
# For processing WhatsApp data file
from whatstk import df_from_txt_whatsapp
from whatstk.data import whatsapp_urls

# Date before which chat is not scored
cut_off_date = 20240506

In [74]:
# file_key = '20231213_townhall_chat'
# file_key = '20231213_beerhall_chat'
# file_key = '20231214_agroverse_chat'
# file_key = '20231214_main_dao_website_chat'
file_key = '20240531_agl_chat'


In [75]:
# Importing WhatsApp data file
df = df_from_txt_whatsapp(filepath=f"./data/{file_key}.txt")
df.head(100)

# Setup work analysis file
analysis_file = open(f"analysis/{file_key}.csv", 'w')
analysis_writer = csv.writer(analysis_file)

header = ['Date', 'Description', 'Entity', 'Amount', 'Currency', 'Transaction Type']
# data = ['20231109', Chat message, 'TrueSight DAO', 'USD', 'Investment']
analysis_writer.writerow(header)

pp = pprint.PrettyPrinter(indent=4)

In [76]:
# For OpenAI client
from openai import OpenAI

# Acquire OpenAI API Key via this URL 
#    https://platform.openai.com/api-keys
# Save Secret in .env file with 
#    OPENAI_API_KEY=YOUR SECRET KEY

openai_client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [77]:
total_rows = df.count()['date']

In [89]:
# Calling OpenAi API to get time taken for work done
def check_agl_transaction(work_description):
    payload = f"Agroverse maintain a chat log of all activities pertaining to our operations\
                   There are primarily the following types of activities in our operations\
                   investor injected capital to finance shipment of cacao nibs\
                   paid supplier for cacao nibs\
                   paid freighting company for transport of cacao nibs\
                   paid customs broker for clearance of cacao nibs\
                   assembled cacao into bags\
                   sold unpackaged cacao\
                   sold packaged 8 ounce cacao nibs\
                   \
                   \
                For each chat log entry if it corresponds to any of the activity type listed above then indicate the corresponding Transaction Type for it\
                \
                   'Investment' if investor injected capital to finance shipment of cacao nibs\
                   'Asset' if paying supplier for cacao nibs\
                   'Asset' if paying freighting company for transport of cacao nibs\
                   'Asset' if paying customs broker for clearance of cacao nibs\
                   'Asset' if assembled cacao into bags\
                   'Asset' if sold unpackaged cacao\
                   'Asset' if sold packaged 8 ounce cacao nibs\
                   'Withdrawal' if investor decided to withdrawl cacao nibs owned by him from Agroverse management\
                   \
                   \
                 These are the types of currency that are recognized\
                    USD\
                    Brazilian Reis\
                    Cacao Nibs KG\
                    Kraft Pouches\
                    8 Ounce Package Craft Pouch\
                    1 Gallon Zip Loc Bags\
                    espresso cup\
                    donation\
                 \
                 Response must be short and concise\
                 Response must be in a single line CSV format. Response must not include headers\
                    Currency 1 (String); Amount 1 (Float); Currency 2 (String); Amount 2 (Float); Transaction Type (String)\
                 \
                 For each transaction, there will likely be up to two different currencies involved\
                   Example 1: Macy bought 1 bag of 8 ounce of cacao nibs for USD $10\
                       we should expect\
                          8 Ounce Package Craft Pouch; -1; USD; 10.00; Assets\
                          \
                   Example 2: Sold 1 pound for usd20 to Chris\
                       we should expect\
                          Cacao Nibs KG; -0.453592; USD; 20.00; Assets\
                   \
                   Example 3: Sold one cup of espresso for one USD\
                       we should expect\
                          Espress cup; -1; USD; 1.00; Assets\
                   \
                   Example 4: Received donation of one USD\
                       we should expect\
                          Unknown; 0; USD; 1.00; Assets\
                   \
                 If Action Description does not make sense return the following values \
                     Unknown; 0; Unknown; 0; Unknown\
                 \
                 If it is not clear what the currency type maps to in the Action description, don't assume anything just return the following values \
                     Unknown; 0; Unknown; 0; Unknown\
                     \
                     Example: Spend  $5.99 for reusable coffee bag\
                         We should expect\
                          Unknown; 0; USD; 5.99; Assets\
                     \
                 For the following 'chat log entry' determine Currency 1, Amount 1, Currency 2, Amount 2 and Transaction Type \
                     \
                 \
                 {work_description}."
    
    work_process_completion = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": payload,
            }
        ],
        model="gpt-3.5-turbo",
    )
    return work_process_completion.choices[0].message.content

# response = check_agl_transaction("Gary Teh walked the dog for 1 hour")

In [92]:
format = '%Y%m%d'
for index, row in df.iterrows():
    # Doesn't process record if occurred before a specific date
    if int(row['date'].strftime(format)) < cut_off_date:
        continue
    
    clean_work_description = row['message'].encode('ascii','ignore') 
    response = check_agl_transaction(clean_work_description)
    openai_result = response.split(';')
    
    currency_1 = "Unknown"
    amount_1 = 0
    currency_2 = "Unknown"
    amount_2 = 0    
    transaction_type = "Unknown"
    
    if len(openai_result) == 5:
        currency_1 = openai_result[0]
        amount_1 = openai_result[1]
        currency_2 = openai_result[2]
        amount_2 = openai_result[3]        
        transaction_type = openai_result[4]

    # header = ['Date', 'Description', 'Entity', 'Amount', 'Currency', 'Transaction Type']
    
    print("\n\n\n")
    print(f"Processing {index} of {total_rows}: ")
    print(row['message'])
    print('Raw results:')
    pp.pprint(response)
    print('Formatted results:')
    pp.pprint(openai_result)
    print('Formatted results length:')
    pp.pprint(len(openai_result))


    if currency_1 == "Unknown" and currency_2 == "Unknown" :
        data_1 = [
            row['date'].strftime(format),
            row['message'], 
            row['username'], 
            amount_1, 
            currency_1, 
            transaction_type
        ]
        print('Row of record written 1:')
        pp.pprint(data_1)
        analysis_writer.writerow(data_1)  
    
    if currency_1 != "Unknown":
        data_1 = [
            row['date'].strftime(format),
            row['message'], 
            row['username'], 
            amount_1, 
            currency_1, 
            transaction_type
        ]
        print('Row of record written 2:')
        pp.pprint(data_1)
        analysis_writer.writerow(data_1)  

    if currency_2 != "Unknown":
        data_2 = [
            row['date'].strftime(format),
            row['message'], 
            row['username'], 
            amount_2, 
            currency_2, 
            transaction_type
        ]
        pp.pprint(data_2)
        analysis_writer.writerow(data_2)  
    
    
          





Processing 7931 of 8118: 
Spend  $5.99 for reusable coffee bag
Raw results:
'Unknown; 0; USD; 5.99; Asset'
Formatted results:
['Unknown', ' 0', ' USD', ' 5.99', ' Asset']
Formatted results length:
5
[   '20240506',
    'Spend  $5.99 for reusable coffee bag',
    'Gary Teh',
    ' 5.99',
    ' USD',
    ' Asset']


ValueError: I/O operation on closed file.

In [ ]:
analysis_file.close()

In [41]:
payload = 'Sold one more bag for usd10'

response = check_agl_transaction(payload)
pp.pprint(response)

'8 Ounce Package Craft Pouch; 1; USD; 10.00; Asset'
